
## *Data Science Unit 4 Sprint 3 Assignment 1*

# Recurrent Neural Networks and Long Short Term Memory (LSTM)

![Monkey at a typewriter](https://upload.wikimedia.org/wikipedia/commons/thumb/3/3c/Chimpanzee_seated_at_typewriter.jpg/603px-Chimpanzee_seated_at_typewriter.jpg)

It is said that [**infinite monkeys typing for an infinite amount of time**](https://en.wikipedia.org/wiki/Infinite_monkey_theorem) will eventually type, among other things, the complete works of William Shakespeare. Let's see if we can get there a bit faster, with the power of Recurrent Neural Networks and LSTM.

We will focus specifically on Shakespeare's Sonnets in order to improve our model's ability to learn from the data.

In [2]:
import random
import sys
import os

import requests
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.callbacks import LambdaCallback

from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, Bidirectional
from tensorflow.keras.layers import LSTM

%matplotlib inline

# a custom data prep class that we'll be using 
from data_cleaning_toolkit_class import data_cleaning_toolkit

### Use request to pull data from a URL

[**Read through the request documentation**](https://requests.readthedocs.io/en/master/user/quickstart/#make-a-request) in order to learn how to download the Shakespeare Sonnets from the Gutenberg website. 

**Protip:** Do not over think it.

In [3]:
# download all of Shakespears Sonnets from the Project Gutenberg website

# here's the link for the sonnets
url_shakespeare_sonnets = "https://www.gutenberg.org/cache/epub/1041/pg1041.txt"

# use request and the url to download all of the sonnets - save the result to `r`
r = requests.get(url_shakespeare_sonnets)

In [4]:
# move the downloaded text out of the request object - save the result to `raw_text_data`
# hint: take at look at the attributes of `r`
raw_text_data = r.text

In [6]:
# check the data type of `raw_text_data`
type(raw_text_data)

str

### Data Cleaning

In [7]:
# as usual, we are tasked with cleaning up messy data
# Question: Do you see any characters that we could use to split up the text?
raw_text_data[:3000]

"\ufeffThe Project Gutenberg EBook of Shakespeare's Sonnets, by William Shakespeare\r\n\r\nThis eBook is for the use of anyone anywhere at no cost and with\r\nalmost no restrictions whatsoever.  You may copy it, give it away or\r\nre-use it under the terms of the Project Gutenberg License included\r\nwith this eBook or online at www.gutenberg.org\r\n\r\n\r\nTitle: Shakespeare's Sonnets\r\n\r\nAuthor: William Shakespeare\r\n\r\nPosting Date: April 7, 2014 [EBook #1041]\r\nRelease Date: September, 1997\r\nLast Updated: March 10, 2010\r\n\r\nLanguage: English\r\n\r\n\r\n*** START OF THIS PROJECT GUTENBERG EBOOK SHAKESPEARE'S SONNETS ***\r\n\r\n\r\n\r\n\r\nProduced by Joseph S. Miller and Embry-Riddle Aeronautical\r\nUniversity Library. HTML version by Al Haines.\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\nTHE SONNETS\r\n\r\nby William Shakespeare\r\n\r\n\r\n\r\n\r\n  I\r\n\r\n  From fairest creatures we desire increase,\r\n  That thereby beauty's rose might never die,\r\n  But as the riper

In [8]:
# split the text into lines and save the result to `split_data`
split_data = raw_text_data.split('\r\n')
split_data

["\ufeffThe Project Gutenberg EBook of Shakespeare's Sonnets, by William Shakespeare",
 '',
 'This eBook is for the use of anyone anywhere at no cost and with',
 'almost no restrictions whatsoever.  You may copy it, give it away or',
 're-use it under the terms of the Project Gutenberg License included',
 'with this eBook or online at www.gutenberg.org',
 '',
 '',
 "Title: Shakespeare's Sonnets",
 '',
 'Author: William Shakespeare',
 '',
 'Posting Date: April 7, 2014 [EBook #1041]',
 'Release Date: September, 1997',
 'Last Updated: March 10, 2010',
 '',
 'Language: English',
 '',
 '',
 "*** START OF THIS PROJECT GUTENBERG EBOOK SHAKESPEARE'S SONNETS ***",
 '',
 '',
 '',
 '',
 'Produced by Joseph S. Miller and Embry-Riddle Aeronautical',
 'University Library. HTML version by Al Haines.',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 'THE SONNETS',
 '',
 'by William Shakespeare',
 '',
 '',
 '',
 '',
 '  I',
 '',
 '  From fairest creatures we desire increase,',
 "  That thereby beaut

In [11]:
# we need to drop all the boilder plate text (i.e. titles and descriptions) as well as white spaces
# so that we are left with only the sonnets themselves 
split_data[:40] 

["\ufeffThe Project Gutenberg EBook of Shakespeare's Sonnets, by William Shakespeare",
 '',
 'This eBook is for the use of anyone anywhere at no cost and with',
 'almost no restrictions whatsoever.  You may copy it, give it away or',
 're-use it under the terms of the Project Gutenberg License included',
 'with this eBook or online at www.gutenberg.org',
 '',
 '',
 "Title: Shakespeare's Sonnets",
 '',
 'Author: William Shakespeare',
 '',
 'Posting Date: April 7, 2014 [EBook #1041]',
 'Release Date: September, 1997',
 'Last Updated: March 10, 2010',
 '',
 'Language: English',
 '',
 '',
 "*** START OF THIS PROJECT GUTENBERG EBOOK SHAKESPEARE'S SONNETS ***",
 '',
 '',
 '',
 '',
 'Produced by Joseph S. Miller and Embry-Riddle Aeronautical',
 'University Library. HTML version by Al Haines.',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 'THE SONNETS',
 '',
 'by William Shakespeare',
 '']

**Use list index slicing in order to remove the titles and descriptions so we are only left with the sonnets.**


In [17]:
# sonnets exists between these indicies 
# titles and descriptions exist outside of these indicies

# use index slicing to isolate the sonnet lines - save the result to `sonnets`
sonnets = split_data[43:]

In [18]:
# notice how all non-sonnet lines have far less characters than the actual sonnet lines?
# well, let's use that observation to filter out all the non-sonnet lines
sonnets[200:240]

['  Since sweets and beauties do themselves forsake',
 '  And die as fast as they see others grow;',
 "    And nothing 'gainst Time's scythe can make defence",
 '    Save breed, to brave him when he takes thee hence.',
 '',
 '  XIII',
 '',
 '  O! that you were your self; but, love you are',
 '  No longer yours, than you your self here live:',
 '  Against this coming end you should prepare,',
 '  And your sweet semblance to some other give:',
 '  So should that beauty which you hold in lease',
 '  Find no determination; then you were',
 "  Yourself again, after yourself's decease,",
 '  When your sweet issue your sweet form should bear.',
 '  Who lets so fair a house fall to decay,',
 '  Which husbandry in honour might uphold,',
 "  Against the stormy gusts of winter's day",
 "  And barren rage of death's eternal cold?",
 '    O! none but unthrifts. Dear my love, you know,',
 '    You had a father: let your son say so.',
 '',
 '  XIV',
 '',
 '  Not from the stars do I my judgement pluck

In [20]:
# any string with less than n_chars characters will be filtered out - save results to `filtered_sonnets`
n_chars = 10
filtered_sonnets = [x for x in sonnets if len(x) > n_chars]

In [21]:
# ok - much better!
# but we still need to remove all the punctuation and case normalize the text
filtered_sonnets

['  From fairest creatures we desire increase,',
 "  That thereby beauty's rose might never die,",
 '  But as the riper should by time decease,',
 '  His tender heir might bear his memory:',
 '  But thou, contracted to thine own bright eyes,',
 "  Feed'st thy light's flame with self-substantial fuel,",
 '  Making a famine where abundance lies,',
 '  Thy self thy foe, to thy sweet self too cruel:',
 "  Thou that art now the world's fresh ornament,",
 '  And only herald to the gaudy spring,',
 '  Within thine own bud buriest thy content,',
 "  And tender churl mak'st waste in niggarding:",
 '    Pity the world, or else this glutton be,',
 "    To eat the world's due, by the grave and thee.",
 '  When forty winters shall besiege thy brow,',
 "  And dig deep trenches in thy beauty's field,",
 "  Thy youth's proud livery so gazed on now,",
 "  Will be a tatter'd weed of small worth held:",
 '  Then being asked, where all thy beauty lies,',
 '  Where all the treasure of thy lusty days;',
 ' 

### Use custom data cleaning tool 

Use one of the methods in `data_cleaning_toolkit` to clean your data.

There is an example of this in the guided project.

In [22]:
# instantiate the data_cleaning_toolkit class - save result to `dctk`

dctk = data_cleaning_toolkit()

In [25]:
# use data_cleaning_toolkit to remove punctuation and to case normalize - save results to `clean_sonnets`
clean_sonnets = [dctk.clean_data(x) for x in filtered_sonnets]

In [26]:
# much better!
clean_sonnets

['from fairest creatures we desire i crease',
 'that thereby beautys rose mightever die',
 'but as the riper should by time decease',
 'his te der heir might bear his memory',
 'but thou co tracted to thi e owbright eyes',
 'feedst thy lights flame with selfsubsta tial fuel',
 'maki g a fami e where abu da ce lies',
 'thy self thy foe to thy sweet self too cruel',
 'thou that artow the worlds fresh or ame t',
 'a d o ly herald to the gaudy spri g',
 'withithi e owbud buriest thy co te t',
 'a d te der churl makst waste iiggardi g',
 'pity the world or else this gluttobe',
 'to eat the worlds due by the grave a d thee',
 'wheforty wi ters shall besiege thy brow',
 'a d dig deep tre ches ithy beautys field',
 'thy youths proud livery so gazed oow',
 'will be a tatterd weed of small worth held',
 'thebei g asked where all thy beauty lies',
 'where all the treasure of thy lusty days',
 'to say withithi e owdeep su keeyes',
 'were aalleati g shame a d thriftless praise',
 'how much more pra

### Use your data tool to create character sequences 

We'll need the `create_char_sequenes` method for this task. However this method requires a parameter call `maxlen` which is responsible for setting the maximum sequence length. 

So what would be a good sequence length, exactly? 

In order to answer that question, let's do some statistics! 

In [33]:
def calc_stats(corpus):
    """
    Calculates statisics on the length of every line in the sonnets
    """
    
    # write a list comprehension that calculates each sonnets line length - save the results to `doc_lens`
    doc_lens = [len(x) for x in clean_sonnets]

    # use numpy to calculate and return the mean, median, std, max, min of the doc lens - all in one line of code
    mean, med, std, max_, min_ = np.mean(doc_lens), np.median(doc_lens), np.std(doc_lens), np.max(doc_lens), np.min(doc_lens)
    
    return mean, med, std, max_, min_

In [34]:
# sonnet line length statistics 
mean ,med, std, max_, min_ = calc_stats(clean_sonnets)
mean, med, std, max_, min_ 

(41.63926380368098, 40.0, 8.474762984176849, 72, 14)

In [35]:
# using the results of the sonnet line length statistics, use your judgement and select a value for maxlen
# use .create_char_sequences() to create sequences
maxlen = 42
dctk.create_char_sequenes(clean_sonnets, maxlen=maxlen)

sequences:  20842


Take a look at the `data_cleaning_toolkit_class.py` file. 

In the first 4 lines of code in the `create_char_sequences` method, class attributes `n_features` and `unique_chars` are created. Let's call them in the cells below.

In [36]:
# number of input features for our LSTM model
dctk.n_features

27

In [40]:
# unique charactes that appear in our sonnets 
dctk.chars

['x',
 'i',
 'h',
 'c',
 'j',
 ' ',
 'q',
 't',
 'y',
 'v',
 'd',
 'o',
 'n',
 'e',
 'a',
 'p',
 'm',
 'b',
 'f',
 's',
 'z',
 'w',
 'r',
 'k',
 'u',
 'l',
 'g']

## Time for Questions 

----
**Question 1:** 

Why are the `number of unique characters` (i.e. **dctk.unique_chars**) and the `number of model input features` (i.e. **dctk.n_features**) the same?

**Hint:** The model that we will shortly be building here is very similar to the text generation model that we built in the guided project.

**Answer 1:**

Because they are the same thing.


**Question 2:**

Take a look at the print out of `dctk.unique_chars` one more time. Notice that there is a white space. 

Why is it desirable to have a white space as a possible character to predict?

**Answer 2:**

If we don't have a whitespace, words cannot be formed. A single string will be formed, with no spaces.

----

### Use our data tool to create X and Y splits

You'll need the `create_X_and_Y` method for this task. 

In [41]:
# TODO: provide a walk through of data_cleaning_toolkit with unit tests that check for understanding 
X, y = dctk.create_X_and_Y()

![](https://miro.medium.com/max/891/0*jGB1CGQ9HdeUwlgB)

In [42]:
# notice that our input matrix isn't actually a matrix - it's a rank 3 tensor
X.shape

(20842, 42, 27)

In $X$.shape we see three numbers (*n1*, *n2*, *n3*). What do these numbers mean?

Well, *n1* tells us the number of samples that we have. But what about the other two?

In [43]:
# first index returns a signle sample, which we can see is a sequence 
first_sample_index = 0 
X[first_sample_index]

array([[False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       ...,
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False]])

Notice that each sequence (i.e. $X[i]$ where $i$ is some index value) is `maxlen` long and has `dctk.n_features` number of features. Let's try to better understand this shape. 

In [44]:
# each sequence is maxlen long and has dctk.n_features number of features
X[first_sample_index].shape

(42, 27)

**Each row corresponds to a character vector** and there are `maxlen` number of character vectors. 

**Each column corresponds to a unique character** and there are `dctk.n_features` number of features. 


In [45]:
# let's index for a single character vector 
first_char_vect_index = 0
X[first_sample_index][first_char_vect_index]

array([False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
        True, False, False, False, False, False, False, False, False])

Notice that there is a single `TRUE` value and all the rest of the values are `FALSE`. 

This is a one-hot encoding for which character appears at each index within a sequence. Specifically, the cell above is looking at the first character in the sequence.

Only a single character can appear as the first character in a sequence, so there will necessarily be a single `TRUE` value and the rest will be `FALSE`. 

Let's say that `TRUE` appears in the $ith$ index; by  $ith$ index we simply mean some index in the general case. How can we find out which character that actually corresponds to?

To answer this question, we need to use the character-to-integer look up dictionaries. 

In [46]:
# take a look at the index to character dictionary
# if a TRUE appears in the 0th index of a character vector,
# then we know that whatever char you see below next to the 0th key 
# is the character that that character vector is endcoding for
dctk.int_char

{0: 'x',
 1: 'i',
 2: 'h',
 3: 'c',
 4: 'j',
 5: ' ',
 6: 'q',
 7: 't',
 8: 'y',
 9: 'v',
 10: 'd',
 11: 'o',
 12: 'n',
 13: 'e',
 14: 'a',
 15: 'p',
 16: 'm',
 17: 'b',
 18: 'f',
 19: 's',
 20: 'z',
 21: 'w',
 22: 'r',
 23: 'k',
 24: 'u',
 25: 'l',
 26: 'g'}

In [47]:
# let's look at an example to tie it all together

seq_len_counter = 0

# index for a single sample 
for seq_of_char_vects in X[first_sample_index]:
    
    # get index with max value, which will be the one TRUE value 
    index_with_TRUE_val = np.argmax(seq_of_char_vects)
    
    print (dctk.int_char[index_with_TRUE_val])
    
    seq_len_counter+=1
    
print ("Sequence length: {}".format(seq_len_counter))

f
r
o
m
 
f
a
i
r
e
s
t
 
c
r
e
a
t
u
r
e
s
 
w
e
 
d
e
s
i
r
e
 
i
 
c
r
e
a
s
e
 
Sequence length: 42


## Time for Questions 

----
**Question 1:** 

In your own words, how would you describe the numbers from the shape print out of `X.shape` to a fellow classmate?


**Answer 1:**

The first number is how many samples we have in the data set--number of observations. The second number is the number of characters there are in each sample. The third number is the number of unique characters that could possibly be in the samples.

----


### Build a Text Generation model

Now that we have prepped our data (and understood that process) let's finally build out our character generation model, similar to what we did in the guided project.

In [48]:
def sample(preds, temperature=1.0):
    """
    Helper function to sample an index from a probability array
    """
    # convert preds to array 
    preds = np.asarray(preds).astype('float64')
    # scale values 
    preds = np.log(preds) / temperature
    # exponentiate values
    exp_preds = np.exp(preds)
    # this equation should look familar to you (hint: it's an activation function)
    preds = exp_preds / np.sum(exp_preds)
    # Draw samples from a multinomial distribution
    probas = np.random.multinomial(1, preds, 1)
    # return the index that corresponds to the max probability 
    return np.argmax(probas)

def on_epoch_end(epoch, _):
    """"
    Function invoked at end of each epoch. Prints the text generated by our model.
    """
    
    print()
    print('----- Generating text after Epoch: %d' % epoch)
    

    # randomly pick a starting index 
    # will be used to take a random sequence of chars from `text`
    start_index = random.randint(0, len(text) - dctk.maxlen - 1)
    
    # this is our seed string (i.e. input seqeunce into the model)
    generated = ''

    # start the sentence at index `start_index` and include the next` dctk.maxlen` number of chars
    sentence = text[start_index: start_index + dctk.maxlen]

    # add to generated
    generated += sentence

    
    print('----- Generating with seed: "' + sentence + '"')
    sys.stdout.write(generated)
    
    # use model to predict what the next 40 chars should be that follow the seed string
    for i in range(40):

        # shape of a single sample in a rank 3 tensor 
        x_dims = (1, dctk.maxlen, dctk.n_features)
        # create an array of zeros with shape x_dims
        # recall that python considers zeros and boolean FALSE as the same
        x_pred = np.zeros(x_dims)

        # create a seq vector for our randomly select sequence 
        # i.e. create a numerical encoding for each char in the sequence 
        for t, char in enumerate(sentence):
            # for sample 0 in seq index t and character `char` encode a 1 (which is the same as a TRUE)
            x_pred[0, t, dctk.char_int[char]] = 1

        # next, take the seq vector and pass into model to get a prediction of what the next char should be 
        preds = model.predict(x_pred, verbose=0)[0]
        # use the sample helper function to get index for next char 
        next_index = sample(preds)
        # use look up dict to get next char 
        next_char = dctk.int_char[next_index]

        # append next char to sequence 
        sentence = sentence[1:] + next_char 
        
        sys.stdout.write(next_char)
        sys.stdout.flush()
    print()

In [49]:
# need this for on_epoch_end()
text = " ".join(clean_sonnets)

In [50]:
# create callback object that will print out text generation at the end of each epoch 
# use for real-time monitoring of model performance
print_callback = LambdaCallback(on_epoch_end=on_epoch_end)

----
### Train Model

Build a text generation model using LSTMs. Feel free to reference the model used in the guided project. 

It is recommended that you train this model to at least 50 epochs (but more if you're computer can handle it). 

You are free to change up the architecture as you wish. 

Just in case you have difficultly training a model, there is a pre-trained model saved to a file called `trained_text_gen_model.h5` that you can load in (the same way that you learned how to load in Keras models in Sprint 2 Module 4). 

In [52]:
# build text generation model layer by layer 
# instantiate model
model = Sequential()

# hidden layer 1 
model.add(LSTM(200, 
               input_shape=(dctk.maxlen, dctk.n_features), # think of input_shape as implicitly declaring the input layer 
               return_sequences=True)) # set to true whenever using 2 or more LSTM layers 

# hidden layer 2 
model.add(LSTM(100))

# this is our output layer
# recall that n_features = num of nodes in output layer 
model.add(Dense(dctk.n_features, 
                activation='softmax'))

# notice that we are using categorical_crossentropy this time around - why?
model.compile(loss='categorical_crossentropy', 
              optimizer='adam')

# fit the model
model.fit(X, y,
          batch_size=256,
          epochs=50,
          callbacks=[print_callback],
          workers=2)

Epoch 1/50
82/82 [==============================] - 22s 235ms/step - loss: 2.9141

----- Generating text after Epoch: 0
----- Generating with seed: "eescore year would make the world away let"
eescore year would make the world away letsrhhecit ssos ryllthrca s   atw e  a ota
Epoch 2/50
82/82 [==============================] - 19s 237ms/step - loss: 2.7402

----- Generating text after Epoch: 1
----- Generating with seed: " will sourly leave her till he have prevai"
 will sourly leave her till he have prevaies ei eet wo vusewdy tefelpt osiue ohtir
Epoch 3/50
82/82 [==============================] - 20s 245ms/step - loss: 2.6202

----- Generating text after Epoch: 2
----- Generating with seed: " solicit co tributio s from states where w"
 solicit co tributio s from states where wou shftt mou tafwti coebk tes hhefre e m
Epoch 4/50
82/82 [==============================] - 19s 230ms/step - loss: 2.4260

----- Generating text after Epoch: 3
----- Generating with seed: "cked to hope what is yo

ific permissioif you doot charge a ythi g ielfeawi g wiss shich de hoves buti g ha
Epoch 31/50
82/82 [==============================] - 19s 232ms/step - loss: 1.2501

----- Generating text after Epoch: 30
----- Generating with seed: "d a very woe before a joy proposd behi d a"
d a very woe before a joy proposd behi d arpeatga t of syatli g ithat sulfess of l
Epoch 32/50
82/82 [==============================] - 20s 247ms/step - loss: 1.1915

----- Generating text after Epoch: 31
----- Generating with seed: "ayst without attai t oerlook the dedicated"
ayst without attai t oerlook the dedicated whothait rod byothare a d io e these so
Epoch 33/50
82/82 [==============================] - 19s 229ms/step - loss: 1.1190

----- Generating text after Epoch: 32
----- Generating with seed: "e t doubti g of the rest love is a babe th"
e t doubti g of the rest love is a babe theellove toom it ygi h wom mis temarh cha
Epoch 34/50
82/82 [==============================] - 19s 228ms/step - loss: 1.0312


In [53]:
# save trained model to file 
model.save("trained_text_gen_model.h5")

### Let's play with our trained model 

Now that we have a trained model that, though far from perfect, is able to generate actual English words, we can take a look at the predictions to continue to learn more about how a text generation model works. 

We can also take this as an opportunity to unpack the `def on_epoch_end` function to better understand how it works. 

In [54]:
# this is our joinned clean sonnet data
text

'from fairest creatures we desire i crease that thereby beautys rose mightever die but as the riper should by time decease his te der heir might bear his memory but thou co tracted to thi e owbright eyes feedst thy lights flame with selfsubsta tial fuel maki g a fami e where abu da ce lies thy self thy foe to thy sweet self too cruel thou that artow the worlds fresh or ame t a d o ly herald to the gaudy spri g withithi e owbud buriest thy co te t a d te der churl makst waste iiggardi g pity the world or else this gluttobe to eat the worlds due by the grave a d thee wheforty wi ters shall besiege thy brow a d dig deep tre ches ithy beautys field thy youths proud livery so gazed oow will be a tatterd weed of small worth held thebei g asked where all thy beauty lies where all the treasure of thy lusty days to say withithi e owdeep su keeyes were aalleati g shame a d thriftless praise how much more praise deservd thy beautys use if thou couldst a swer this fair child of mi e shall sum my c

In [58]:
# randomly pick a starting index 
# will be used to take a random sequence of chars from `text`
# run this cell a few times and you'll see `start_index` is random
start_index = random.randint(0, len(text) - dctk.maxlen - 1)
start_index

49051

In [59]:
# next use the randomly selected starting index to sample a sequence from the `text`

# this is our seed string (i.e. input seqeunce into the model)
generated = ''

# start the sentence at index `start_index` and include the next` dctk.maxlen` number of chars
sentence = text[start_index: start_index + dctk.maxlen]

# add to generated
generated += sentence

generated

'll determi ate for how do i hold thee but '

In [60]:
# this block of code let's us know what the seed string is 
# i.e. the input seqeunce into the model
print('----- Generating with seed: "' + sentence + '"')
sys.stdout.write(generated)

----- Generating with seed: "ll determi ate for how do i hold thee but "
ll determi ate for how do i hold thee but 

In [61]:
# use model to predict what the next 40 chars should be that follow the seed string
for i in range(40):

    # shape of a single sample in a rank 3 tensor 
    x_dims = (1, dctk.maxlen, dctk.n_features)
    # create an array of zeros with shape x_dims
    # recall that python considers zeros and boolean FALSE as the same
    x_pred = np.zeros(x_dims)

    # create a seq vector for our randomly select sequence 
    # i.e. create a numerical encoding for each char in the sequence 
    for t, char in enumerate(sentence):
        # for sample 0 in seq index t and character `char` encode a 1 (which is the same as a TRUE)
        x_pred[0, t, dctk.char_int[char]] = 1

    # next, take the seq vector and pass into model to get a prediction of what the next char should be 
    preds = model.predict(x_pred, verbose=0)[0]
    # use the sample helper function to get index for next char 
    next_index = sample(preds)
    # use look up dict to get next char 
    next_char = dctk.int_char[next_index]

    # append next char to sequence 
    sentence = sentence[1:] + next_char 

In [62]:
# this is the seed string
generated

'll determi ate for how do i hold thee but '

In [63]:
# these are the 40 chars that the model thinks should come after the seed stirng
sentence

't thei g a d that which to ofor sull wirgh'

In [64]:
# how put it all together
generated + sentence

'll determi ate for how do i hold thee but t thei g a d that which to ofor sull wirgh'

# Resources and Stretch Goals

## Stretch goals:
- Refine the training and generation of text to be able to ask for different genres/styles of Shakespearean text (e.g. plays versus sonnets)
- Train a classification model that takes text and returns which work of Shakespeare it is most likely to be from
- Make it more performant! Many possible routes here - lean on Keras, optimize the code, and/or use more resources (AWS, etc.)
- Revisit the news example from class, and improve it - use categories or tags to refine the model/generation, or train a news classifier
- Run on bigger, better data

## Resources:
- [The Unreasonable Effectiveness of Recurrent Neural Networks](https://karpathy.github.io/2015/05/21/rnn-effectiveness/) - a seminal writeup demonstrating a simple but effective character-level NLP RNN
- [Simple NumPy implementation of RNN](https://github.com/JY-Yoon/RNN-Implementation-using-NumPy/blob/master/RNN%20Implementation%20using%20NumPy.ipynb) - Python 3 version of the code from "Unreasonable Effectiveness"
- [TensorFlow RNN Tutorial](https://github.com/tensorflow/models/tree/master/tutorials/rnn) - code for training a RNN on the Penn Tree Bank language dataset
- [4 part tutorial on RNN](http://www.wildml.com/2015/09/recurrent-neural-networks-tutorial-part-1-introduction-to-rnns/) - relates RNN to the vanishing gradient problem, and provides example implementation
- [RNN training tips and tricks](https://github.com/karpathy/char-rnn#tips-and-tricks) - some rules of thumb for parameterizing and training your RNN